In [8]:
# The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
# Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

import logging
import sys
import warnings
from urllib.parse import urlparse

import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


models = {
    'Random Forest Regressor': RandomForestRegressor(n_estimators=100),
    'Gradient Boosting Regressor': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1),
    'Linear Regression': LinearRegression(),
    'SVR': SVR(C=1, kernel='rbf'),
    'Decision Tree Regressor': DecisionTreeRegressor(max_depth=10),
    'ElasticNet': ElasticNet(alpha=0.6, l1_ratio=0.4, random_state=42)
}

if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha =  0.9
    l1_ratio =  0.4
    experiment_id = mlflow.create_experiment('multiple_model')

    with mlflow.start_run(experiment_id=experiment_id):
        for model_name, model in models.items():
    # Train the model on the full training set
            model.fit(train_x, train_y)

    # Make predictions on the test set
            y_pred = model.predict(test_x)



            (rmse, mae, r2) = eval_metrics(test_y, y_pred)

            print(f"  RMSE: {rmse}")
            print(f"  MAE: {mae}")
            print(f"  R2: {r2}")

        # mlflow.log_param("alpha", alpha)
        # mlflow.log_param("l1_ratio", l1_ratio)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            predictions = model.predict(train_x)
            signature = infer_signature(train_x, predictions)

            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
            if tracking_url_type_store != "file":
            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(
                    model, "model", registered_model_name="ElasticnetWineModel", signature=signature
            )
            else:
                mlflow.sklearn.log_model(model, "model", signature=signature)

  RMSE: 0.580523901316733
  MAE: 0.4145
  R2: 0.5224993801140592
  RMSE: 0.6086983721839029
  MAE: 0.47286995409365945
  R2: 0.4750257400739144
  RMSE: 0.6603811224461621
  MAE: 0.5073644199888925
  R2: 0.38209312191101574
  RMSE: 0.7718660798607456
  MAE: 0.5762460441467399
  R2: 0.15585385535107532
  RMSE: 0.8140366797038331
  MAE: 0.522592817581464
  R2: 0.061094943993141304
  RMSE: 0.7928446872861473
  MAE: 0.626666444473971
  R2: 0.10934405701835759
